In [1]:
import pandas as pd
import numpy as np
from scipy import stats


from dash_website.utils.graphs import heatmap_by_sorted_dimensions, add_custom_legend_axis
from dash_website.utils.graphs import heatmap_by_sorted_dimensions, add_custom_legend_axis
from dash_website import (
    ORDER_DIMENSIONS,
    CUSTOM_ORDER,
)

subset_method = "union"
correlation_type = "pearson"
first_category = "Genetics"
second_category = "Phenotypic"

data_comparison_upper = pd.read_feather(f"../../all_data/xwas/univariate_correlations/correlations/categories/correlations_{first_category}.feather").to_dict()
data_comparison_lower = pd.read_feather(f"../../all_data/xwas/univariate_correlations/correlations/categories/correlations_{second_category}.feather").to_dict()

In [2]:
def get_table_and_customdata(data_comparison, subset_method, correlation_type):
    correlations_raw = pd.DataFrame(data_comparison).set_index(
        ["dimension_1", "subdimension_1", "r2_1", "r2_std_1", "dimension_2", "subdimension_2", "r2_2", "r2_std_2"]
    )
    correlations_raw.columns = pd.MultiIndex.from_tuples(
        list(map(eval, correlations_raw.columns.tolist())), names=["subset_method", "correlation_type"]
    )
    correlations = correlations_raw[[(subset_method, correlation_type), (subset_method, "number_variables")]]
    correlations.columns = ["correlation", "number_variables"]
    correlations.reset_index(inplace=True)

    table_correlations = correlations.pivot(
        index=["dimension_1", "subdimension_1"],
        columns=["dimension_2", "subdimension_2"],
        values="correlation",
    ).loc[ORDER_DIMENSIONS, ORDER_DIMENSIONS]

    customdata_list = []
    for customdata_item in ["r2_1", "r2_std_1", "r2_2", "r2_std_2", "number_variables"]:
        customdata_list.append(
            correlations.pivot(
                index=["dimension_1", "subdimension_1"],
                columns=["dimension_2", "subdimension_2"],
                values=customdata_item,
            )
            .loc[ORDER_DIMENSIONS, ORDER_DIMENSIONS]
            .values
        )
    stacked_customdata = list(map(list, np.dstack(customdata_list)))

    customdata = pd.DataFrame(None, index=ORDER_DIMENSIONS, columns=ORDER_DIMENSIONS)
    customdata[customdata.columns] = stacked_customdata

    return table_correlations, customdata, correlations

In [3]:
import plotly.graph_objs as go

table_correlations_upper, customdata_upper, correlations_upper = get_table_and_customdata(
    data_comparison_upper, subset_method, correlation_type
)
np.fill_diagonal(table_correlations_upper.values, np.nan)

table_correlations_lower, customdata_lower, correlations_lower = get_table_and_customdata(
    data_comparison_lower, subset_method, correlation_type
)
np.fill_diagonal(table_correlations_lower.values, np.nan)

fig_points = go.Figure()

hovertemplate_points = "Dimension 1: %{customdata[0]}, Subdimension 1: %{customdata[1]}<br>Dimenions 2: %{customdata[2]}, Subdimension 2: %{customdata[3]}<Br>Correlation first category %{x:.3f}, Correlation second category %{y:.3f}<extra></extra>"

y_points = correlations_lower.set_index(["dimension_1", "subdimension_1", "dimension_2", "subdimension_2"])[
    "correlation"
]
y_points[
    (y_points.index.get_level_values("dimension_1") == y_points.index.get_level_values("dimension_2"))
    & (y_points.index.get_level_values("subdimension_1") == y_points.index.get_level_values("subdimension_2"))
] = np.nan

x_points = correlations_upper.set_index(["dimension_1", "subdimension_1", "dimension_2", "subdimension_2"]).loc[
    y_points.index, "correlation"
]
x_points[
    (x_points.index.get_level_values("dimension_1") == x_points.index.get_level_values("dimension_2"))
    & (x_points.index.get_level_values("subdimension_1") == x_points.index.get_level_values("subdimension_2"))
] = np.nan

customdata_points = list(map(list, y_points.index.values))

fig_points = go.Figure()
fig_points.add_scatter(x=[-1.1, 1.1], y=[-1.1, 1.1], mode="lines", name="perfect correlation line")

fig_points.add_scatter(
    x=x_points.values,
    y=y_points.values,
    mode="markers",
    customdata=customdata_points,
    hovertemplate=hovertemplate_points,
    marker={"size": 5},
    name=f"{first_category} vs. {second_category} correlation for one aging dimension",
)
correlation_of_correlations, p_value_of_correlations = stats.pearsonr(
    x_points[x_points.notna() & y_points.notna()], y_points[x_points.notna() & y_points.notna()]
)

fig_points.update_layout(
    yaxis={"title": f"{second_category} correlation", "range": [-1.1, 1.1], "showgrid": False},
    xaxis={"title": f"{first_category} correlation", "range": [-1.1, 1.1], "showgrid": False},
    width=1500,
    height=1500,
)

sorted_dimensions = (
    correlations_upper.set_index(["dimension_1", "subdimension_1"]).loc[CUSTOM_ORDER].index.drop_duplicates()
)

triangular_heatmap_values = np.triu(table_correlations_upper)
triangular_heatmap_values += np.tril(table_correlations_lower, k=1)
triangular_heatmap = pd.DataFrame(
    triangular_heatmap_values, index=table_correlations_upper.index, columns=table_correlations_upper.columns
)

customdata_triangular_values = np.triu(customdata_upper)
customdata_triangular_values += np.tril(customdata_lower, k=1)
customdata_triangular = pd.DataFrame(
    customdata_triangular_values, index=table_correlations_upper.index, columns=table_correlations_upper.columns
)

sorted_triangular_heatmap = triangular_heatmap.loc[sorted_dimensions, sorted_dimensions]
sorted_customdata_triangular = customdata_triangular.loc[sorted_dimensions, sorted_dimensions]

hovertemplate_triangular = "Correlation: %{z:.3f} <br><br>Dimensions 1: %{x} <br>R2: %{customdata[0]:.3f} +- %{customdata[1]:.3f} <br>Dimensions 2: %{y}<br>R2: %{customdata[2]:.3f} +- %{customdata[3]:.3f} <br>Number variables: %{customdata[4]}<br><extra></extra>"

fig_triangular = heatmap_by_sorted_dimensions(
    sorted_triangular_heatmap, hovertemplate_triangular, sorted_customdata_triangular
)
fig_triangular = add_custom_legend_axis(
    fig_triangular, sorted_triangular_heatmap, size_dimension=14, size_subdimension=12
)
fig_triangular.update_layout(
    yaxis={"title": f"{second_category} correlation", "showgrid": False, "zeroline": False},
    xaxis={"title": f"{first_category} correlation", "showgrid": False, "zeroline": False},
    width=1500,
    height=1500,
)

difference_heatmap = table_correlations_upper - table_correlations_lower

sorted_difference_heatmap = difference_heatmap.loc[sorted_dimensions, sorted_dimensions]
sorted_customdata_upper = customdata_upper.loc[sorted_dimensions, sorted_dimensions]

hovertemplate_difference = "Difference in correlation: %{z:.3f} <br><br>Dimensions 1: %{x} <br>R2: %{customdata[0]:.3f} +- %{customdata[1]:.3f} <br>Dimensions 2: %{y}<br>R2: %{customdata[2]:.3f} +- %{customdata[3]:.3f}<br><extra></extra>"

fig_difference = heatmap_by_sorted_dimensions(
    sorted_difference_heatmap, hovertemplate_difference, sorted_customdata_upper, zmin=-2, zmax=2
)
fig_difference = add_custom_legend_axis(
    fig_difference, sorted_difference_heatmap, size_dimension=14, size_subdimension=12
)
fig_difference.update_layout(
    yaxis={"showgrid": False, "zeroline": False},
    xaxis={"showgrid": False, "zeroline": False},
    width=1500,
    height=1500,
)

None

In [18]:
sorted_triangular_heatmap["Brain"]

subdimension_1                             *  Cognitive       MRI
dimension_1      subdimension_1                                  
*                *                       NaN        NaN       NaN
*instances01     *                       NaN        NaN       NaN
*instances1.5x   *                 -0.191328  -0.087364 -0.396896
*instances23     *                  0.710316   0.397386  0.768987
Brain            *                       NaN   0.714376  0.957501
                 Cognitive          0.410920        NaN  0.130928
                 MRI                0.948826   0.112195       NaN
Eyes             *                  0.099258   0.015905  0.093376
                 All                     NaN        NaN       NaN
                 Fundus             0.084013   0.041154  0.083549
                 OCT                0.094527   0.034298  0.086943
Hearing          *                  0.149490   0.329436  0.061987
Lungs            *                  0.114753   0.094100  0.098149
Arterial         *                       NaN        NaN       NaN
                 Carotids                NaN        NaN       NaN
                 PulseWaveAnalysis       NaN        NaN       NaN
Heart            *                  0.193211   0.053716  0.197435
                 ECG                0.101371   0.076676  0.088058
                 MRI                0.211142   0.057294  0.206539
Abdomen          *                       NaN        NaN       NaN
                 Liver                   NaN        NaN       NaN
                 Pancreas                NaN        NaN       NaN
Musculoskeletal  *                  0.211377   0.076543  0.205293
                 FullBody           0.198710   0.068771  0.191920
                 Hips               0.158972   0.064086  0.166382
                 Knees              0.104944   0.057599  0.105000
                 Scalars            0.131631   0.109746  0.116209
                 Spine              0.158276   0.057015  0.156116
PhysicalActivity *                  0.135100   0.154476  0.110560
Biochemistry     *                       NaN        NaN       NaN
                 Blood                   NaN        NaN       NaN
                 Urine                   NaN        NaN       NaN
BloodCells       *                  0.203388   0.016190  0.099080

In [21]:
triangular_heatmap #.loc[("Arterial", "*"), "Brain"]

* *instances01 *instances1.5x  \
                                           *            *              *   
*                *                       NaN          NaN            NaN   
*instances01     *                  0.838413          NaN       0.995208   
*instances1.5x   *                  0.998501     0.008787            NaN   
*instances23     *                  0.774242     0.551438       0.180503   
Abdomen          *                  0.375136     0.368087       0.155090   
                 Liver              0.342234     0.339481       0.134956   
                 Pancreas           0.320634     0.319480       0.143712   
Arterial         *                  0.356929     0.283191       0.109654   
                 Carotids           0.338114     0.255188       0.084547   
                 PulseWaveAnalysis  0.222601     0.330039       0.128030   
Biochemistry     *                  0.330020     0.395290       0.105152   
                 Blood              0.299999     0.372617       0.083323   
                 Urine              0.146211     0.163325       0.137610   
BloodCells       *                  0.158290     0.167142       0.118501   
Brain            *                  0.383839     0.235570       0.135881   
                 Cognitive          0.145770     0.261460       0.155807   
                 MRI                0.362098     0.208357       0.111118   
Eyes             *                  0.723633     0.856213       0.065231   
                 All                     NaN          NaN            NaN   
                 Fundus             0.618413     0.708825       0.070602   
                 OCT                0.509078     0.595617       0.085064   
Hearing          *                  0.154660     0.243065       0.219207   
Heart            *                  0.564943     0.321027       0.099185   
                 ECG                0.211535     0.159469       0.107615   
                 MRI                0.574109     0.314912       0.100737   
Lungs            *                  0.161773     0.230451       0.113460   
Musculoskeletal  *                  0.573588     0.658467       0.162645   
                 FullBody           0.534690     0.619282       0.159793   
                 Hips               0.316388     0.432793       0.137932   
                 Knees              0.294252     0.315935       0.111269   
                 Scalars            0.172699     0.233288       0.126686   
                 Spine              0.390372     0.492587       0.107767   
PhysicalActivity *                  0.998352     0.015686       0.999851   

                                   *instances23   Abdomen                      \
                                              *         *     Liver  Pancreas   
*                *                          NaN       NaN       NaN       NaN   
*instances01     *                          NaN       NaN       NaN       NaN   
*instances1.5x   *                          NaN       NaN -0.197093  0.671523   
*instances23     *                          NaN  1.502254  1.000000  0.966220   
Abdomen          *                     0.502254       NaN  1.851430  0.960883   
                 Liver                 0.447073  0.883755       NaN  1.389253   
                 Pancreas              0.428799  0.860370  0.525806       NaN   
Arterial         *                     0.423142  0.200691  0.178282  0.173652   
                 Carotids              0.394605  0.207675  0.187425  0.177024   
                 PulseWaveAnalysis     0.189965  0.084459  0.070353  0.074557   
Biochemistry     *                     0.111406  0.135350  0.107201  0.116886   
                 Blood                 0.142350  0.123717  0.102877  0.106026   
                 Urine                 0.075094  0.059176  0.052985  0.048583   
BloodCells       *                     0.093021  0.085543  0.091827  0.070970   
Brain            *                     0.513838  0.164394  0.153142  0.136555   
     